# Libaray

In [1]:
import os ,time

import glob
import random
from PIL import Image, ImageFile
import ttach as tta
import matplotlib.pyplot as plt
import imutils

import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, grad_scaler
from torch.optim.lr_scheduler import _LRScheduler
import torch_optimizer as optim
from efficientnet_pytorch import EfficientNet

import numpy as np
import pandas as pd, cv2
from tqdm import tqdm
from easydict import EasyDict
from sklearn.preprocessing import LabelEncoder #Label Encoder로 숫자로 변경함
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from ptflops import get_model_complexity_info

from collections import Counter
import argparse
import neptune.new as neptune
import timm

#안하면 오류남
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore')

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device setting : {device}")

device setting : cuda


In [3]:
test_df = pd.read_csv('/home/user/effcientNet/explosive/merge_RP_explosive_test/test_df.csv')

In [4]:
args = EasyDict({'encoder_name':'efficientnet-b1',
                 'drop_path_rate':0.2,
                 'num_classes':5,
                 'bad' : False
                })

In [5]:
def get_train_augmentation(img_size, ver):
    if ver==1:
        transform = transforms.Compose([
            transforms.ToTensor(), 
            transforms.Resize((img_size, img_size)),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
                ])      
    return transform

In [6]:
transform = get_train_augmentation(img_size = 224, ver = 1)

In [7]:
class Network(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.encoder = EfficientNet.from_pretrained(args.encoder_name,num_classes = args.num_classes )
        
    def forward(self, x):
        x = self.encoder(x)
        return x

In [8]:
class Test_Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.file_name = df['file_name'].values
        self.transform = transform

        print(f"테스트 데이터셋 생성 완료,,\n 데이터셋 사이즈 : {len(self.file_name)}")

    def __getitem__(self, index):        
        image = np.array(Image.open(f'{self.file_name[index]}').convert('RGB'))

        if self.transform is not None:
            image = self.transform(Image.fromarray(image))

        return image

    def __len__(self):
        return len(self.file_name)

In [9]:

test_dataset = Test_Dataset(test_df, transform)
test_load = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

테스트 데이터셋 생성 완료,,
 데이터셋 사이즈 : 3290


In [10]:
def predict(args, test_load, model_path):
    _transforms = tta.Compose([
        tta.Rotate90(angles=[0,20]),
    ])
    model = Network(args).to(device)
    model.load_state_dict(torch.load(model_path)['state_dict'])
    model = tta.ClassificationTTAWrapper(model, _transforms).to(device)
    model.eval()

    output = []
    pred = []
    with torch.no_grad():
        with autocast():
            for batch in tqdm(test_load):
                images = torch.tensor(batch, dtype = torch.float32, device = device).clone().detach()
                preds = model(images)
                pred.extend(preds)
                output.extend(torch.tensor(torch.argmax(preds, dim=1), dtype=torch.int32).cpu().numpy())
    
    return  output, pred

In [11]:
model_path = "./result/001/new/efficientnet-b1_best_model.pth"
args = EasyDict({'encoder_name':'efficientnet-b1',
                 'drop_path_rate':0.2,
                 'num_classes':5,
                })
output1, preds1 = predict(args, test_load, model_path)

Loaded pretrained weights for efficientnet-b1


100%|███████████████████████████████████████████| 52/52 [00:05<00:00,  8.75it/s]


In [12]:
answer_labels = test_df['label']
submit_labels = output1

In [13]:
print(f1_score(answer_labels,submit_labels,average='macro'))
print(accuracy_score(answer_labels,submit_labels))
print(len(answer_labels))

0.8798833558305059
0.9048632218844985
3290


In [14]:
'''
for i in range(len(answer_labels)):
    
    if answer_labels[i] != submit_labels[i]:
        print(f'정답 : {answer_labels[i]}-> 예측 : {submit_labels[i]}')
        print(f"Wrong Answer : {test_df['file_name'][i]}")
        image = cv2.imread(test_df['file_name'][i])
        image = imutils.resize(image, width=800)
        cv2.imshow(f'correct : {answer_labels[i]}-> predict : {submit_labels[i]}',image)
        cv2.waitKey()
        cv2.destroyAllWindows()
'''

'\nfor i in range(len(answer_labels)):\n    \n    if answer_labels[i] != submit_labels[i]:\n        print(f\'정답 : {answer_labels[i]}-> 예측 : {submit_labels[i]}\')\n        print(f"Wrong Answer : {test_df[\'file_name\'][i]}")\n        image = cv2.imread(test_df[\'file_name\'][i])\n        image = imutils.resize(image, width=800)\n        cv2.imshow(f\'correct : {answer_labels[i]}-> predict : {submit_labels[i]}\',image)\n        cv2.waitKey()\n        cv2.destroyAllWindows()\n'

In [15]:
'''
for i in range(len(answer_labels)):
    print(f'정답 : {answer_labels[i]}-> 예측 : {submit_labels[i]}')
    print(f"Wrong Answer : {test_df['file_name'][i]}")
'''

'\nfor i in range(len(answer_labels)):\n    print(f\'정답 : {answer_labels[i]}-> 예측 : {submit_labels[i]}\')\n    print(f"Wrong Answer : {test_df[\'file_name\'][i]}")\n'

In [16]:
cnt = 1
aa = 0
range_f1=[]
for i in range(len(answer_labels)):
    #print(answer_labels[i])
    if answer_labels[i] != submit_labels[i]:
        print(f'정답 : {answer_labels[i]}-> 예측 : {submit_labels[i]}')
        #print(f"Wrong Answer : {test_df['file_name'][i]}")
        if answer_labels[i] != aa:
            aa += 1
            range_f1.append(cnt-1)
            cnt = 1
        print(cnt)
        cnt += 1
range_f1.append(cnt-1)

정답 : 0-> 예측 : 1
1
정답 : 0-> 예측 : 1
2
정답 : 0-> 예측 : 3
3
정답 : 0-> 예측 : 1
4
정답 : 0-> 예측 : 1
5
정답 : 0-> 예측 : 1
6
정답 : 0-> 예측 : 3
7
정답 : 0-> 예측 : 4
8
정답 : 0-> 예측 : 1
9
정답 : 0-> 예측 : 2
10
정답 : 0-> 예측 : 3
11
정답 : 0-> 예측 : 3
12
정답 : 0-> 예측 : 3
13
정답 : 0-> 예측 : 1
14
정답 : 0-> 예측 : 3
15
정답 : 0-> 예측 : 3
16
정답 : 0-> 예측 : 3
17
정답 : 0-> 예측 : 4
18
정답 : 0-> 예측 : 1
19
정답 : 0-> 예측 : 4
20
정답 : 0-> 예측 : 3
21
정답 : 0-> 예측 : 3
22
정답 : 0-> 예측 : 3
23
정답 : 0-> 예측 : 1
24
정답 : 0-> 예측 : 1
25
정답 : 0-> 예측 : 3
26
정답 : 0-> 예측 : 3
27
정답 : 0-> 예측 : 3
28
정답 : 0-> 예측 : 1
29
정답 : 0-> 예측 : 1
30
정답 : 0-> 예측 : 2
31
정답 : 0-> 예측 : 4
32
정답 : 0-> 예측 : 3
33
정답 : 0-> 예측 : 3
34
정답 : 0-> 예측 : 1
35
정답 : 0-> 예측 : 1
36
정답 : 0-> 예측 : 1
37
정답 : 0-> 예측 : 4
38
정답 : 0-> 예측 : 1
39
정답 : 0-> 예측 : 3
40
정답 : 0-> 예측 : 3
41
정답 : 0-> 예측 : 3
42
정답 : 0-> 예측 : 1
43
정답 : 0-> 예측 : 3
44
정답 : 0-> 예측 : 3
45
정답 : 0-> 예측 : 4
46
정답 : 0-> 예측 : 1
47
정답 : 0-> 예측 : 3
48
정답 : 0-> 예측 : 1
49
정답 : 0-> 예측 : 3
50
정답 : 0-> 예측 : 1
51
정답 : 0-> 예측 : 4
52
정답 : 0-> 예측 : 1
53
정답

In [17]:
print(len(range_f1))
print(range_f1)
range_list=[809,207,754,766,754]
answer_f1=[]
predict_f1=[]
for r in range(5):
    for i in range(range_list[r]):
        answer_f1.append(1)
    #print(answer_f1)
    #print(len(answer_f1))

    #predict_f1=[]
    #answer_f1=[]
    #for a in range(len(range_f1)):
    
    for i in range(range_f1[r]):
        predict_f1.append(0)
    for i in range(range_list[r]-range_f1[r]):
        predict_f1.append(1)
        #print(predict_f1)
    #print(len(predict_f1))
    print(r+1)
    print("f1_score :", round(f1_score(answer_f1,predict_f1),3))
    print("acc:",round(accuracy_score(answer_f1,predict_f1),3))
    print("error_rate:",round(1-accuracy_score(answer_f1,predict_f1),3))
    predict_f1.clear()
    answer_f1.clear()


5
[107, 61, 21, 52, 72]
1
f1_score : 0.929
acc: 0.868
error_rate: 0.132
2
f1_score : 0.827
acc: 0.705
error_rate: 0.295
3
f1_score : 0.986
acc: 0.972
error_rate: 0.028
4
f1_score : 0.965
acc: 0.932
error_rate: 0.068
5
f1_score : 0.95
acc: 0.905
error_rate: 0.095


In [18]:
'''
ccc=[16,29,10,12]
print(len(ccc))
print(ccc)
answer_f1_c=[]
for i in range(100):
    answer_f1_c.append(1)
#print(answer_f1)
#print(len(answer_f1))

predict_f1_c=[]
for a in range(len(ccc)):
    
    for i in range(ccc[a]):
        predict_f1_c.append(0)
    for i in range(100-ccc[a]):
        predict_f1_c.append(1)
    #print(predict_f1)
    #print(len(predict_f1))
    print("\nf1_score :", f1_score(answer_f1_c,predict_f1_c))
    print(a)
    predict_f1_c.clear()
'''

'\nccc=[16,29,10,12]\nprint(len(ccc))\nprint(ccc)\nanswer_f1_c=[]\nfor i in range(100):\n    answer_f1_c.append(1)\n#print(answer_f1)\n#print(len(answer_f1))\n\npredict_f1_c=[]\nfor a in range(len(ccc)):\n    \n    for i in range(ccc[a]):\n        predict_f1_c.append(0)\n    for i in range(100-ccc[a]):\n        predict_f1_c.append(1)\n    #print(predict_f1)\n    #print(len(predict_f1))\n    print("\nf1_score :", f1_score(answer_f1_c,predict_f1_c))\n    print(a)\n    predict_f1_c.clear()\n'